In [3]:
### preparing model for extracted features from the payloads

import pandas as pd


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier , VotingClassifier
from sklearn.model_selection import GridSearchCV

# appending path for importing common process ua

import sys 
sys.path.append('/home/th3h04x/Documents/github/ProjectHeatBlast')


from ReverseProxy.utils.util import process_ua


payloads_with_feature_df = pd.read_csv("build/combined_payloads_with_features.csv" , index_col=0)
payloads_df = pd.read_csv("build/combined_payloads.csv" , index_col=0)


In [4]:
#dropping payload column

payloads_with_feature_df.drop(["payloads"],axis=1,inplace=True)

payloads_df

,payloads,is_malicious
0,Nokia,0
1,Sony,0
2,BMW,0
3,Palm,0
4,Adobe,0
...,...,...
41575,<form><button formaction=javascript&colon;aler...,1
41576,"<math><a xlink:href=""//jsfiddle.net/t846h/"">click",1
41577,"<object data=data:text/html;base64,PHN2Zy9vbmx...",1
41578,"<iframe src=""data:text/html,%3C%73%63%72%69%70...",1


In [32]:
import ipyparallel as ipp

cluster = ipp.Cluster(n=5)

await cluster.start_cluster()

Starting 5 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


<Cluster(cluster_id='1640844115-ndtu', profile='default', controller=<running>, engine_sets=['1640844116'])>

In [5]:
# getting x and y values

x = payloads_with_feature_df.drop(["is_malicious"],axis=1).values 
y = payloads_with_feature_df["is_malicious"].values


#Dont Run take long time to run

# Hyper Paramater Tuning for each custom feature models

In [34]:
# k neigherest neighbours


knn_clf = GridSearchCV(KNeighborsClassifier() , {
        'n_neighbors':[10,50,100,150,200,250,500]
        } ,
        cv=5)

knn_clf.fit(x,y)
pd.DataFrame(knn_clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.050770,0.028584,0.219265,0.024089,10,{'n_neighbors': 10},0.997715,0.998918,0.996994,0.999519,0.996873,0.998004,0.001050,1
1,0.032193,0.000457,0.265185,0.014796,50,{'n_neighbors': 50},0.996633,0.998557,0.996272,0.999639,0.997475,0.997715,0.001243,2
2,0.033726,0.000493,0.346127,0.011961,100,{'n_neighbors': 100},0.996633,0.997956,0.994829,0.999399,0.995671,0.996898,0.001627,3
3,0.035850,0.003823,0.487658,0.041852,150,{'n_neighbors': 150},0.996513,0.997835,0.994228,0.999158,0.995430,0.996633,0.001736,4
4,0.034459,0.002850,0.544248,0.034341,200,{'n_neighbors': 200},0.995551,0.997354,0.993146,0.999158,0.995190,0.996080,0.002039,5
5,0.035396,0.001770,0.620059,0.022432,250,{'n_neighbors': 250},0.994589,0.997114,0.992304,0.998918,0.994949,0.995575,0.002263,6
6,0.034745,0.001867,1.158534,0.044737,500,{'n_neighbors': 500},0.991582,0.996753,0.990741,0.997715,0.993506,0.994060,0.002760,7


In [35]:
#Multinomial naive bayes

mnb_clf = GridSearchCV( MultinomialNB() , {
    "alpha":[0.1,0.3,0.5,0.7,1,2]
},cv=5)

mnb_clf.fit(x,y)
pd.DataFrame(mnb_clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.007337,0.001768,0.000980,0.000146,0.1,{'alpha': 0.1},0.990981,0.990741,0.984367,0.992424,0.990139,0.989731,0.002785,1
1,0.006927,0.000734,0.001040,0.000445,0.3,{'alpha': 0.3},0.990981,0.990741,0.984367,0.992424,0.990139,0.989731,0.002785,1
2,0.007764,0.002791,0.001890,0.000982,0.5,{'alpha': 0.5},0.990981,0.990741,0.984367,0.992424,0.990139,0.989731,0.002785,1
3,0.008651,0.002145,0.001306,0.000574,0.7,{'alpha': 0.7},0.990981,0.990741,0.984367,0.992424,0.990139,0.989731,0.002785,1
4,0.008467,0.002017,0.000886,0.000207,1,{'alpha': 1},0.990981,0.990741,0.984367,0.992424,0.990139,0.989731,0.002785,1
5,0.006779,0.001401,0.000901,0.000140,2,{'alpha': 2},0.990981,0.990741,0.984367,0.992424,0.990019,0.989707,0.002781,6


In [36]:
#support vector machine

svc_clf = GridSearchCV(SVC() , {
    "kernel":["rbf","linear","sigmoid"] ,
    "gamma":[0.1,0.5,1,2],
    "C":[1,10,20]
},cv=5)

svc_clf.fit(x,y)
pd.DataFrame(svc_clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,16.980585,4.636666,1.410972,0.095776,1,0.1,rbf,"{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}",0.998677,0.998196,0.992785,0.996873,0.993025,0.995911,0.002526,13
1,0.184056,0.023023,0.014990,0.002141,1,0.1,linear,"{'C': 1, 'gamma': 0.1, 'kernel': 'linear'}",0.998316,0.999278,0.994829,1.000000,0.998196,0.998124,0.001775,9
2,29.776442,1.850892,6.121400,0.112542,1,0.1,sigmoid,"{'C': 1, 'gamma': 0.1, 'kernel': 'sigmoid'}",0.538841,0.538841,0.538841,0.538841,0.538841,0.538841,0.000000,25
3,36.966296,4.848876,2.320946,0.155137,1,0.5,rbf,"{'C': 1, 'gamma': 0.5, 'kernel': 'rbf'}",0.994108,0.994589,0.980159,0.985209,0.979918,0.986797,0.006451,18
4,0.172707,0.025158,0.014844,0.002490,1,0.5,linear,"{'C': 1, 'gamma': 0.5, 'kernel': 'linear'}",0.998316,0.999278,0.994829,1.000000,0.998196,0.998124,0.001775,9
5,23.891106,1.137041,5.181720,0.054879,1,0.5,sigmoid,"{'C': 1, 'gamma': 0.5, 'kernel': 'sigmoid'}",0.538841,0.538841,0.538841,0.538841,0.538841,0.538841,0.000000,25
6,27.303234,2.952930,2.200237,0.173753,1,1,rbf,"{'C': 1, 'gamma': 1, 'kernel': 'rbf'}",0.991101,0.991101,0.970418,0.973184,0.972463,0.979654,0.009391,21
7,0.175486,0.026662,0.014720,0.002086,1,1,linear,"{'C': 1, 'gamma': 1, 'kernel': 'linear'}",0.998316,0.999278,0.994829,1.000000,0.998196,0.998124,0.001775,9
8,21.919056,0.613030,5.102172,0.070922,1,1,sigmoid,"{'C': 1, 'gamma': 1, 'kernel': 'sigmoid'}",0.538841,0.538841,0.538841,0.538841,0.538841,0.538841,0.000000,25
9,25.212657,1.957793,2.353222,0.183446,1,2,rbf,"{'C': 1, 'gamma': 2, 'kernel': 'rbf'}",0.986051,0.984728,0.955868,0.956590,0.952742,0.967196,0.014917,24


In [37]:
#Decision Tree Classifier

dt_clf = GridSearchCV(DecisionTreeClassifier(),{
    "criterion":["gini","entropy"],
    "max_depth":[10,20,30]
},cv=5)

dt_clf.fit(x,y)

pd.DataFrame(dt_clf.cv_results_)



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.024136,0.002845,0.000952,0.000150,gini,10,"{'criterion': 'gini', 'max_depth': 10}",0.996873,0.999038,0.993386,0.999399,0.989658,0.995671,0.003689,3
1,0.022837,0.002610,0.000819,0.000084,gini,20,"{'criterion': 'gini', 'max_depth': 20}",0.996873,0.999038,0.993386,0.999399,0.991101,0.995960,0.003236,1
2,0.022425,0.002061,0.000863,0.000151,gini,30,"{'criterion': 'gini', 'max_depth': 30}",0.996873,0.999038,0.993386,0.999399,0.986291,0.994998,0.004850,5
3,0.015447,0.000959,0.000822,0.000243,entropy,10,"{'criterion': 'entropy', 'max_depth': 10}",0.997114,0.999158,0.993386,0.999278,0.986291,0.995046,0.004868,4
4,0.015769,0.000969,0.000758,0.000048,entropy,20,"{'criterion': 'entropy', 'max_depth': 20}",0.996873,0.999158,0.993386,0.999278,0.984848,0.994709,0.005373,6
5,0.016071,0.001259,0.000713,0.000036,entropy,30,"{'criterion': 'entropy', 'max_depth': 30}",0.996994,0.999038,0.993386,0.999278,0.991101,0.995960,0.003218,1


In [38]:
#random forest classifier

rf_clf = GridSearchCV(RandomForestClassifier(),{
    "criterion":["gini","entropy"],
    "max_depth":[10,20,30] ,
    "n_estimators":[20,30,50]
},cv=5)

rf_clf.fit(x,y)
pd.DataFrame(rf_clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.142943,0.009453,0.007293,0.000251,gini,10,20,"{'criterion': 'gini', 'max_depth': 10, 'n_esti...",0.997114,0.998918,0.993386,0.999639,0.996392,0.997090,0.002193,12
1,0.207602,0.001799,0.010155,0.000581,gini,10,30,"{'criterion': 'gini', 'max_depth': 10, 'n_esti...",0.996994,0.999158,0.993506,0.999639,0.997595,0.997379,0.002166,1
2,0.344065,0.015548,0.016312,0.000526,gini,10,50,"{'criterion': 'gini', 'max_depth': 10, 'n_esti...",0.997114,0.999038,0.993506,0.999639,0.996633,0.997186,0.002159,7
3,0.135736,0.006622,0.006742,0.000098,gini,20,20,"{'criterion': 'gini', 'max_depth': 20, 'n_esti...",0.997114,0.998918,0.993386,0.999639,0.997715,0.997354,0.002173,3
4,0.208265,0.010870,0.010244,0.000394,gini,20,30,"{'criterion': 'gini', 'max_depth': 20, 'n_esti...",0.996753,0.999038,0.993506,0.999639,0.996994,0.997186,0.002155,7
5,0.368043,0.015037,0.017062,0.001042,gini,20,50,"{'criterion': 'gini', 'max_depth': 20, 'n_esti...",0.996994,0.998677,0.993506,0.999639,0.995310,0.996825,0.002222,18
6,0.149692,0.006221,0.007247,0.000438,gini,30,20,"{'criterion': 'gini', 'max_depth': 30, 'n_esti...",0.996994,0.999038,0.993146,0.999639,0.996392,0.997042,0.002295,13
7,0.213646,0.009106,0.010826,0.000585,gini,30,30,"{'criterion': 'gini', 'max_depth': 30, 'n_esti...",0.996994,0.999038,0.993627,0.999639,0.996272,0.997114,0.002143,9
8,0.352192,0.018675,0.016809,0.000534,gini,30,50,"{'criterion': 'gini', 'max_depth': 30, 'n_esti...",0.996994,0.999038,0.993386,0.999639,0.997234,0.997258,0.002186,5
9,0.131916,0.007578,0.007136,0.000520,entropy,10,20,"{'criterion': 'entropy', 'max_depth': 10, 'n_e...",0.997114,0.999038,0.993506,0.999639,0.995070,0.996873,0.002323,15


In [10]:
#logistic regresion

lg_clf = GridSearchCV(LogisticRegression(max_iter=1000) , {
    "C":[1,5,10,20,30]
} ,cv=5)

lg_clf.fit(x,y)
pd.DataFrame(lg_clf.cv_results_)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.183399,0.029281,0.001146,0.000209,1,{'C': 1},0.998437,0.999278,0.994829,1.0,0.997234,0.997956,0.001814,5
1,0.245943,0.031006,0.001083,0.000122,5,{'C': 5},0.998316,0.999278,0.994829,1.0,0.998196,0.998124,0.001775,4
2,0.182175,0.014723,0.001011,0.000331,10,{'C': 10},0.998316,0.999278,0.995551,1.0,0.998196,0.998268,0.001511,3
3,0.207300,0.009337,0.000829,0.000006,20,{'C': 20},0.998196,0.999278,0.995791,1.0,0.998196,0.998292,0.001426,1
4,0.271035,0.093831,0.001090,0.000354,30,{'C': 30},0.998196,0.999278,0.995791,1.0,0.998196,0.998292,0.001426,1


In [7]:
# nltk models using count vectorizer


from sklearn.metrics import accuracy_score, confusion_matrix
#save the model
import pickle , joblib

payloads_df.dropna(inplace=True)

x = payloads_df["payloads"].values 
y = payloads_df['is_malicious'].values


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

cv = CountVectorizer(analyzer=process_ua)
ld = LinearDiscriminantAnalysis()
pca = PCA()


In [8]:
from sklearn.pipeline import Pipeline , make_pipeline

knn_cv_pipe = Pipeline([("count-vectorizer" , cv) ,("knn" ,KNeighborsClassifier()) ])
mnb_cv_pipe = Pipeline([("count-vectorizer" ,cv ), ("mnb" ,MultinomialNB()) ])
svc_cv_pipe = Pipeline([("count-vectorizer" , cv ), ("svc" ,SVC()) ])
dtc_cv_pipe = Pipeline([("count-vectorizer" , cv ), ("dt" ,DecisionTreeClassifier()) ])
rf_cv_pipe = Pipeline([("count-vectorizer" ,cv ), ("rf" ,RandomForestClassifier()) ])
lg_cv_pipe = Pipeline([("count-vectorizer" , cv ), ("lg" ,LogisticRegression(max_iter=1000)) ])

In [36]:
# k neighrest neighbours

knn_pipe_clf = GridSearchCV(knn_cv_pipe , {
    "knn__n_neighbors":[10,50,100,200]
},cv=5)

knn_pipe_clf.fit(x , y)
pd.DataFrame(knn_pipe_clf.cv_results_)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_knn__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,4.140005,0.391701,3.347361,0.070429,10,{'knn__n_neighbors': 10},0.474988,0.466691,0.460919,0.461159,0.508358,0.474423,0.017720,4
1,4.194086,0.226961,3.590007,0.267465,50,{'knn__n_neighbors': 50},0.461881,0.460798,0.461159,0.740019,0.461215,0.517014,0.111503,3
2,4.236153,0.127853,3.571283,0.082662,100,{'knn__n_neighbors': 100},0.614719,0.650673,0.664622,0.602453,0.636079,0.633709,0.022745,1
3,4.277808,0.077537,3.665114,0.063439,200,{'knn__n_neighbors': 200},0.551106,0.560005,0.560606,0.554113,0.555382,0.556242,0.003601,2


In [37]:
#Multinomial naive bayes

mnb_pipe_clf = GridSearchCV( mnb_cv_pipe , {
    "mnb__alpha":[0.1,0.3,0.5,0.7,1,2]
},cv=5)

mnb_pipe_clf.fit(x,y)
pd.DataFrame(mnb_pipe_clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_mnb__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,4.365420,0.272044,1.058808,0.056967,0.1,{'mnb__alpha': 0.1},0.861472,0.892136,0.921477,0.915825,0.862057,0.890593,0.025515,1
1,4.283663,0.175717,1.054741,0.099319,0.3,{'mnb__alpha': 0.3},0.861472,0.892136,0.921477,0.915825,0.862057,0.890593,0.025515,1
2,4.211620,0.105346,1.034671,0.075571,0.5,{'mnb__alpha': 0.5},0.861111,0.892136,0.921477,0.915825,0.862057,0.890521,0.025598,3
3,4.164208,0.085230,1.041637,0.102646,0.7,{'mnb__alpha': 0.7},0.861111,0.892136,0.921477,0.915825,0.862057,0.890521,0.025598,3
4,4.176977,0.081910,1.052407,0.122613,1,{'mnb__alpha': 1},0.860991,0.892136,0.921356,0.915825,0.862057,0.890473,0.025597,5
5,4.069495,0.173100,1.033279,0.045824,2,{'mnb__alpha': 2},0.860991,0.892015,0.921356,0.915825,0.861816,0.890401,0.025649,6


In [45]:
#support vector machine

svc_pipe_clf = GridSearchCV(svc_cv_pipe , {
    "svc__kernel":["rbf","linear","sigmoid"] ,
    "svc__gamma":[0.1,0.5,1,2],
    "svc__C":[1,10,20]
},cv=5)

svc_pipe_clf.fit(x,y)
pd.DataFrame(svc_pipe_clf.cv_results_)

In [44]:
#Decision Tree Classifier

dt_pipe_clf = GridSearchCV(dtc_cv_pipe,{
    "dt__criterion":["gini","entropy"],
    "dt__max_depth":[10,20,30]
},cv=5)

dt_pipe_clf.fit(x,y)

pd.DataFrame(dt_pipe_clf.cv_results_)



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_dt__criterion,param_dt__max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,3.960197,0.058753,0.971637,0.078925,gini,10,"{'dt__criterion': 'gini', 'dt__max_depth': 10}",0.550625,0.563372,0.568663,0.566859,0.563440,0.562592,0.006318,5
1,4.008407,0.122599,0.970091,0.066752,gini,20,"{'dt__criterion': 'gini', 'dt__max_depth': 20}",0.558081,0.574435,0.581169,0.583213,0.575947,0.574569,0.008855,3
2,4.159201,0.089395,1.035278,0.136263,gini,30,"{'dt__criterion': 'gini', 'dt__max_depth': 30}",0.568062,0.579846,0.591871,0.593795,0.593987,0.585512,0.010173,1
3,4.329664,0.072073,1.059689,0.110089,entropy,10,"{'dt__criterion': 'entropy', 'dt__max_depth': 10}",0.550625,0.563372,0.568663,0.566859,0.563440,0.562592,0.006318,5
4,4.245213,0.236425,1.051165,0.100665,entropy,20,"{'dt__criterion': 'entropy', 'dt__max_depth': 20}",0.558081,0.573954,0.580928,0.583093,0.575947,0.574401,0.008799,4
5,4.243208,0.193723,1.006482,0.055594,entropy,30,"{'dt__criterion': 'entropy', 'dt__max_depth': 30}",0.567701,0.579606,0.591871,0.593795,0.593987,0.585392,0.010324,2


In [43]:
#random forest classifier

rf_pipe_clf = GridSearchCV(rf_cv_pipe,{
    "rf__criterion":["gini","entropy"],
    "rf__max_depth":[10,20,30] ,
    "rf__n_estimators":[20,30,50]
},cv=5)

rf_pipe_clf.fit(x,y)
pd.DataFrame(rf_pipe_clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rf__criterion,param_rf__max_depth,param_rf__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,4.550346,0.236006,1.080477,0.086196,gini,10,20,"{'rf__criterion': 'gini', 'rf__max_depth': 10,...",0.565296,0.572631,0.583694,0.583574,0.570655,0.575170,0.007316,14
1,4.320208,0.079138,1.030857,0.076954,gini,10,30,"{'rf__criterion': 'gini', 'rf__max_depth': 10,...",0.556878,0.574194,0.573473,0.568543,0.563921,0.567402,0.006437,15
2,4.321419,0.107835,1.051020,0.075551,gini,10,50,"{'rf__criterion': 'gini', 'rf__max_depth': 10,...",0.555315,0.553752,0.567581,0.564935,0.558749,0.560066,0.005373,18
3,4.114158,0.077065,1.011122,0.079086,gini,20,20,"{'rf__criterion': 'gini', 'rf__max_depth': 20,...",0.588865,0.611953,0.635883,0.629630,0.615394,0.616345,0.016331,6
4,4.160813,0.069819,1.007501,0.073452,gini,20,30,"{'rf__criterion': 'gini', 'rf__max_depth': 20,...",0.587302,0.607383,0.613155,0.621934,0.609381,0.607831,0.011413,9
5,4.237947,0.052994,1.013831,0.088039,gini,20,50,"{'rf__criterion': 'gini', 'rf__max_depth': 20,...",0.585017,0.605099,0.617003,0.609788,0.617198,0.606821,0.011820,10
6,4.125213,0.127767,1.006345,0.071607,gini,30,20,"{'rf__criterion': 'gini', 'rf__max_depth': 30,...",0.583454,0.625301,0.624218,0.659331,0.617318,0.621924,0.024154,3
7,4.251513,0.084445,1.031982,0.075677,gini,30,30,"{'rf__criterion': 'gini', 'rf__max_depth': 30,...",0.587181,0.610991,0.648870,0.618807,0.611064,0.615383,0.019831,7
8,4.406556,0.077587,1.047684,0.080175,gini,30,50,"{'rf__criterion': 'gini', 'rf__max_depth': 30,...",0.592352,0.620731,0.627706,0.641294,0.617438,0.619904,0.016027,4
9,4.164492,0.077288,1.022853,0.089087,entropy,10,20,"{'rf__criterion': 'entropy', 'rf__max_depth': ...",0.571068,0.573954,0.580327,0.574555,0.584486,0.576878,0.004847,13


In [42]:
#logistic regresion

lg_pipe_clf = GridSearchCV(lg_cv_pipe , {
    "lg__C":[1,5,10,20,30]
} ,cv=5)

lg_pipe_clf.fit(x,y)
pd.DataFrame(lg_pipe_clf.cv_results_)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_lg__C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,5.272570,0.282520,1.194869,0.101276,1,{'lg__C': 1},0.766474,0.811328,0.864478,0.838384,0.791461,0.814425,0.034396,5
1,5.692976,0.439154,1.149383,0.088399,5,{'lg__C': 5},0.805195,0.854257,0.889971,0.871813,0.820445,0.848336,0.031492,4
2,5.797869,0.233541,1.143903,0.082128,10,{'lg__C': 10},0.806638,0.855580,0.891053,0.875541,0.821888,0.850140,0.031747,3
3,5.730008,0.319722,1.159975,0.068005,20,{'lg__C': 20},0.809163,0.864358,0.898749,0.884680,0.823933,0.856177,0.034473,2
4,5.719505,0.416561,1.081566,0.070728,30,{'lg__C': 30},0.818062,0.864598,0.899110,0.884800,0.833073,0.859929,0.030495,1


In [7]:
# Count Vectorizer Classification Report

from sklearn.metrics import  classification_report

x_train , x_test , y_train , y_test = train_test_split(x,y,random_state=1)


In [25]:
print("\n\n[Count Vectorizer Results]\n")
lg_cv_pipe.fit(x_train,y_train)
y_pred = lg_cv_pipe.predict(x_test)

print(classification_report(y_test ,y_pred))



[Count Vectorizer Results]

              precision    recall  f1-score   support

           0       1.00      0.71      0.83      4854
           1       0.80      1.00      0.89      5541

    accuracy                           0.86     10395
   macro avg       0.90      0.85      0.86     10395
weighted avg       0.89      0.86      0.86     10395



In [16]:
#support vector classifier

svc_cv_pipe.fit(x_train , y_train)
y_pred = svc_cv_pipe.predict(x_test)
print(classification_report(y_pred , y_test))

              precision    recall  f1-score   support

           0       0.67      0.98      0.80      3312
           1       0.99      0.78      0.87      7083

    accuracy                           0.84     10395
   macro avg       0.83      0.88      0.83     10395
weighted avg       0.89      0.84      0.85     10395



In [17]:
dtc_cv_pipe.fit(x_train , y_train)
y_pred = dtc_cv_pipe.predict(x_test)
print(classification_report(y_pred , y_test))


              precision    recall  f1-score   support

           0       1.00      0.53      0.69      9152
           1       0.22      0.99      0.36      1243

    accuracy                           0.59     10395
   macro avg       0.61      0.76      0.53     10395
weighted avg       0.91      0.59      0.65     10395



In [26]:
#Random Forest 

print("Count Vectorizer Method\n")
rf_cv_pipe.fit(x_train , y_train)
y_pred = rf_cv_pipe.predict(x_test)
print(classification_report(y_pred , y_test))


Count Vectorizer Method

              precision    recall  f1-score   support

           0       1.00      0.53      0.69      9158
           1       0.22      1.00      0.36      1237

    accuracy                           0.59     10395
   macro avg       0.61      0.76      0.53     10395
weighted avg       0.91      0.59      0.65     10395



In [20]:
#Multinomial Naive Bayes

mnb_cv_pipe.fit(x_train , y_train)
y_pred = mnb_cv_pipe.predict(x_test)
print(classification_report(y_test , y_pred))



              precision    recall  f1-score   support

           0       1.00      0.85      0.92      4854
           1       0.89      1.00      0.94      5541

    accuracy                           0.93     10395
   macro avg       0.94      0.93      0.93     10395
weighted avg       0.94      0.93      0.93     10395



In [21]:
#k nearest neighbours

knn_cv_pipe.fit(x_train , y_train)
y_pred = knn_cv_pipe.predict(x_test)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.48      1.00      0.65      4854
           1       1.00      0.06      0.12      5541

    accuracy                           0.50     10395
   macro avg       0.74      0.53      0.39     10395
weighted avg       0.76      0.50      0.37     10395



In [15]:
x = payloads_with_feature_df.drop(["is_malicious"],axis=1).values 
y = payloads_with_feature_df["is_malicious"].values
x_train , x_test , y_train , y_test = train_test_split(x,y,random_state=1 , test_size=0.3)

In [16]:
#Custom Feature Results

# K nearest neighbours

model = KNeighborsClassifier(n_neighbors=10)
model.fit(x_train,y_train)

y_pred = model.predict(x_test)
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5786
           1       1.00      1.00      1.00      6688

    accuracy                           1.00     12474
   macro avg       1.00      1.00      1.00     12474
weighted avg       1.00      1.00      1.00     12474



In [17]:
#svc

model = SVC(kernel="linear",gamma=20,C=2)
model.fit(x_train,y_train)

y_pred = model.predict(x_test)
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5786
           1       1.00      1.00      1.00      6688

    accuracy                           1.00     12474
   macro avg       1.00      1.00      1.00     12474
weighted avg       1.00      1.00      1.00     12474



In [19]:
#Multinomial Naive Bayes

model = MultinomialNB(alpha=0.1)
model.fit(x_train,y_train)

y_pred = model.predict(x_test)
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      5786
           1       0.98      1.00      0.99      6688

    accuracy                           0.99     12474
   macro avg       0.99      0.99      0.99     12474
weighted avg       0.99      0.99      0.99     12474



In [20]:
# DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=30 , criterion="gini")
model.fit(x_train,y_train)

y_pred = model.predict(x_test)
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5786
           1       1.00      1.00      1.00      6688

    accuracy                           1.00     12474
   macro avg       1.00      1.00      1.00     12474
weighted avg       1.00      1.00      1.00     12474



In [21]:
#Randomforest Classifier

model = RandomForestClassifier(max_depth=30 , criterion="entropy" , n_estimators=30)
model.fit(x_train,y_train)

y_pred = model.predict(x_test)
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5786
           1       1.00      1.00      1.00      6688

    accuracy                           1.00     12474
   macro avg       1.00      1.00      1.00     12474
weighted avg       1.00      1.00      1.00     12474

